In [2]:
import pandas as pd
import random

# 1. Obtener ejemplos de calles

Para crear nuestro set de entrenamiento, utilizaremos las calles de la ciudad de Madrid. Así, crearemos ejemplos con nombres reales que más tarde ayudarán a generalizar al modelo.

Descargamos los datos del callejero de Madrid desde [aquí](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=b3c41f3cf6a6c410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default)


In [4]:
df_callejero = pd.read_csv('../data/VialesVigentes_20241226.csv', sep = ';', encoding = 'cp1252')

Una vez cargados los datos, los visualizamos para entender con qué información estamos trabajando. Nos encontramos con 15 columnas, de las cuales solo nos interesan 3: VIA_CLASE, VIA_PAR y VIA_NOMBRE.

In [5]:
df_callejero.head(5)

,COD_VIA,VIA_CLASE,VIA_PAR,VIA_NOMBRE,VIA_NOMBRE_ACENTOS,COD_VIA_COMIENZA,CLASE_COMIENZA,PARTICULA_COMIENZA,NOMBRE_COMIENZA,NOMBRE_ACENTOS_COMIENZA,COD_VIA_TERMINA,CLASE_TERMINA,PARTICULA_TERMINA,NOMBRE_TERMINA,NOMBRE_ACENTOS_TERMINA
0,31001337,AUTOVÍA,NaN,A-1,A-1,31001349,AUTOVÍA,NaN,M-30,M-30,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
1,31001336,AUTOVÍA,NaN,A-2,A-2,310200,CALLE,DE,FRANCISCO SILVELA,FRANCISCO SILVELA,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
2,31001342,AUTOVÍA,NaN,A-3,A-3,480800,PLAZA,DE,MARIANO DE CAVIA,MARIANO DE CAVIA,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
3,31001334,AUTOVÍA,NaN,A-4,A-4,31001349,AUTOVÍA,NaN,M-30,M-30,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL
4,31001341,AUTOVÍA,NaN,A-42,A-42,468400,AVENIDA,DEL,MANZANARES,MANZANARES,99000003,LUGAR,NaN,LIMITE TERMINO MUNICIPAL,LÍMITE TÉRMINO MUNICIPAL


Una vez seleccionadas las columnas de interés, crearemos una columna adicional que sea la concatenación de la partícula y el nombre de la vía, ya que no estamos interesados de que nuestro modelo detecte las partículas de forma distinta al nombre de la vía.

Así mismo, eliminaremos los registros que contengan vacíos.

In [9]:
df_calles = df_callejero[df_callejero.VIA_CLASE != 'AUTOVÍA'][['VIA_CLASE', 'VIA_PAR', 'VIA_NOMBRE']]

df_calles['NOMBRE_VIA'] = df_calles['VIA_PAR'] + ' ' + df_calles['VIA_NOMBRE']
df_calles.drop(columns = ['VIA_PAR', 'VIA_NOMBRE'], inplace  = True)


df_calles.rename(columns={'VIA_CLASE': 'TIPO_VIA'}, inplace = True)
df_calles.dropna(inplace =  True)

In [10]:
df_calles.head(5)

,TIPO_VIA,NOMBRE_VIA
7,CALLE,DEL ABAD JUAN CATALAN
8,CALLE,DE LA ABADA
9,CALLE,DE LOS ABADES
10,CALLE,DE LA ABADESA
11,CALLE,DE ABALOS


# 2. Generar direcciones ficticias

Para terminar de crear las direcciones ficticias, añadiremos más señas a las calles de Madrid. Para ello, de manera aleatoria, incluiremos información sobre el número y la combinación piso-letra

In [7]:
def generar_datos_entrenamiento(df, n):
    """
    Genera N líneas aleatorias a partir de un DataFrame base.
    
    Args:
        df (pd.DataFrame): DataFrame base para seleccionar líneas.
        n (int): Número de líneas aleatorias a generar.
    
    Returns:
        pd.DataFrame: DataFrame con N líneas aleatorias generadas.
    """
    lineas = []
    for _ in range(n):
        # Seleccionar una línea aleatoria del DataFrame base
        row = df.sample(1).copy()

        # Agregar un número aleatorio
        pre_numero = random.choice(['n*', 'nr', 'nº', '', 'n'])
        numero = random.randint(1, 50)
        post_numero = random.choice(['', '', '', ','])
        row['NUMERO'] = f'{pre_numero}{numero}{post_numero}'

        # Agregar combinación aleatoria de piso y letra
        piso = random.randint(1, 10)
        letra = random.choice(['A', 'B', 'C', 'D', 'IZDA', 'DCHA', 'CTRO'])
        espacio = random.choice([' ', '*', 'º', '', '-'])
        combinacion = f"{piso}{espacio}{letra}"
        row['RESTO'] = combinacion

        # Agregar la línea generada a la lista
        lineas.append(row)

    # Combinar todas las líneas en un nuevo DataFrame
    return pd.concat(lineas, ignore_index=True)


In [11]:
train_set = generar_datos_entrenamiento(df_calles, 3000)

# 3. Estructurar los datos para el modelo

Una vez terminamos de generar las direcciones ficticias, daremos forma a nuestros datos para poder entrenar el modelo de spacy.

In [75]:
def crear_entidades(row):
    texto = row['texto']
    entidades = []
    start = 0
    for col, label in zip(['TIPO_VIA', 'NOMBRE_VIA', 'NUMERO', 'RESTO'], ['TIPO_VIA', 'NOMBRE_VIA', 'NUMERO', 'RESTO']):
        value = str(row[col])
        start = texto.find(value, start)  # Buscar el índice inicial de la entidad
        if start != -1:
            end = start + len(value)  # Índice final de la entidad
            entidades.append((start, end, label))
            start = end  # Actualizar el inicio para evitar errores en textos repetidos
    return (texto, {'entities': entidades})

In [76]:
train_set['texto'] = train_set['TIPO_VIA'] + ' ' + train_set['NOMBRE_VIA'] + ' ' + train_set['NUMERO'].astype(str) + ' ' + train_set['RESTO']
train_set['entidades'] = train_set.apply(crear_entidades, axis=1)

In [77]:
train_set.to_csv('./data/train_set.csv', sep = ';', encoding='utf-8-sig', index = None)